In [1]:
import pathlib
import pandas as pd

In [3]:
csv_data_dir = pathlib.Path.cwd() / "ml-25m"
csv_data_dir.exists()

True

In [4]:
tags_genome = csv_data_dir / "genome-tags.csv"
movies = csv_data_dir / "movies.csv"
movie_tags = csv_data_dir / "genome-scores.csv"
links = csv_data_dir / "links.csv"
ratings = csv_data_dir / "ratings.csv"
tags = csv_data_dir / "tags.csv"

tags_genome = pd.read_csv(tags_genome)
movies = pd.read_csv(movies)
movie_tags = pd.read_csv(movie_tags)
links = pd.read_csv(links)
ratings = pd.read_csv(ratings)
tags = pd.read_csv(tags)

In [7]:
for df in [tags_genome, movies, movie_tags, links, ratings, tags]:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15584448 entries, 0 to 15584447
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 356.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423

In [14]:
ratings["rating_id"] = [i for i in range(len(ratings))]
ratings.set_index("rating_id", inplace=True)
ratings.drop(labels="timestamp", inplace=True, axis=1)

In [16]:
ratings.head()

,userId,movieId,rating
rating_id,,,
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [28]:
genre_list = ["Action",
"Adventure",
"Animation",
"Children",
"Comedy",
"Crime",
"Documentary",
"Drama",
"Fantasy",
"Film-Noir",
"Horror",
"Musical",
"Mystery",
"Romance",
"Sci-Fi",
"Thriller",
"War",
"Western",
"IMAX",
"(no genres listed)"]
genre_ids = [i for i in range(len(genre_list))]
genre_df = pd.DataFrame({"genre_id": genre_ids, "genre_name": genre_list})
genre_df.set_index("genre_id", inplace=True)
genre_df.head()

,genre_name
genre_id,
0,Action
1,Adventure
2,Animation
3,Children
4,Comedy


In [29]:
tables = {
    "movies": movies,
    "genres": genre_df,
    "users": pd.DataFrame({"user_id": []}),
    "tag_options": tags_genome,
    "ratings": ratings,
    "movie_genres": pd.DataFrame({"movie_id": [], "genre_id": []}),
    "movie_tags": movie_tags,
    "tag_instances": tags,
}

In [30]:
tables["movies"].head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [31]:
mg = {"movie_id": [], "genre_id": []}

def add_mg_row(movie_id, glist):
    genres = glist.split("|")
    for g in genres:
        try:
            gid = genre_list.index(g)
        except ValueError:
            print(g)
            continue
        mg["movie_id"].append(movie_id)
        mg["genre_id"].append(gid)

for row in tables["movies"].itertuples():
    add_mg_row(row.movieId, row.genres)

In [34]:
len(mg["movie_id"])

112307

In [35]:
tables["movie_genre"] = pd.DataFrame(mg)

In [37]:
useridlist = list(set(list(tables["ratings"]["userId"])))

In [39]:
tables["users"] = pd.DataFrame({"user_id": sorted(useridlist)})

In [40]:
tables["users"].head()

,user_id
0,1
1,2
2,3
3,4
4,5


In [47]:
c = tables["movies"].join(links.set_index("movieId"), on="movieId")
c.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0


In [48]:
tables["movies"] = c

In [51]:
for name, df in tables.items():
    p = csv_data_dir.parent / "processed_csvs"
    assert p.exists()
    p = p / f"{name}.csv"
    df.to_csv(p)